[View in Colaboratory](https://colab.research.google.com/github/Amiralae/modulation-recognition-for-wireless-signals/blob/master/Residual_CNN/ResNet.ipynb)

((110000, 2, 128), [2, 128])


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:59: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(50, (1, 1), padding="valid", kernel_initializer="glorot_uniform", data_format="channels_first", name="res1a_branch2a")`
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:65: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(50, (1, 7), padding="valid", kernel_initializer="glorot_uniform", data_format="channels_first", name="res1a_branch2b")`
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:70: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(50, (1, 3), padding="valid", kernel_initializer="glorot_uniform", data_format="channels_first", name="res1a_branch2c")`
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:74: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(50, (1, 1), kernel_initializer="glorot_uniform", data_format="channels_first", name="res1a_branch1")`
/usr/local

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1, 2, 128)    0                                            
__________________________________________________________________________________________________
zero_padding2d_7 (ZeroPadding2D (None, 1, 2, 132)    0           input_2[0][0]                    
__________________________________________________________________________________________________
res1a_branch2a (Conv2D)         (None, 50, 2, 132)   100         zero_padding2d_7[0][0]           
__________________________________________________________________________________________________
activation_8 (Activation)       (None, 50, 2, 132)   0           res1a_branch2a[0][0]             
__________________________________________________________________________________________________
dropout_9 

 - 42s - loss: 2.2842 - acc: 0.1337 - val_loss: 2.0836 - val_acc: 0.2399
Epoch 7/100
 - 42s - loss: 2.1740 - acc: 0.1676 - val_loss: 1.9330 - val_acc: 0.2394
Epoch 8/100
 - 42s - loss: 2.1235 - acc: 0.1821 - val_loss: 1.9157 - val_acc: 0.2714
Epoch 9/100
 - 42s - loss: 2.1142 - acc: 0.1846 - val_loss: 1.9085 - val_acc: 0.2635
Epoch 10/100
 - 42s - loss: 2.0929 - acc: 0.1942 - val_loss: 2.5682 - val_acc: 0.2121
Epoch 11/100
 - 41s - loss: 2.0374 - acc: 0.2154 - val_loss: 2.0206 - val_acc: 0.2368
Epoch 12/100
 - 42s - loss: 1.9967 - acc: 0.2345 - val_loss: 1.8400 - val_acc: 0.2755
Epoch 13/100
 - 41s - loss: 1.9752 - acc: 0.2467 - val_loss: 1.9086 - val_acc: 0.2803
Epoch 14/100
 - 42s - loss: 1.9628 - acc: 0.2537 - val_loss: 1.8149 - val_acc: 0.2899
Epoch 15/100
 - 41s - loss: 1.9646 - acc: 0.2516 - val_loss: 1.8446 - val_acc: 0.2992
Epoch 16/100
 - 41s - loss: 1.9524 - acc: 0.2603 - val_loss: 1.8207 - val_acc: 0.3006
Epoch 17/100
 - 41s - loss: 1.9396 - acc: 0.2667 - val_loss: 1.7796 - 

Epoch 40/100
 - 42s - loss: 1.7798 - acc: 0.3256 - val_loss: 1.6190 - val_acc: 0.3854
Epoch 41/100
 - 42s - loss: 1.7778 - acc: 0.3276 - val_loss: 1.6578 - val_acc: 0.3751
Epoch 42/100
 - 42s - loss: 1.7747 - acc: 0.3272 - val_loss: 1.6628 - val_acc: 0.3641
Epoch 43/100
 - 42s - loss: 1.7686 - acc: 0.3289 - val_loss: 1.6546 - val_acc: 0.3660
Epoch 44/100
 - 42s - loss: 1.7648 - acc: 0.3273 - val_loss: 1.6163 - val_acc: 0.3900
Epoch 45/100
 - 42s - loss: 1.7484 - acc: 0.3368 - val_loss: 1.6029 - val_acc: 0.3930
Epoch 46/100
 - 42s - loss: 1.7455 - acc: 0.3411 - val_loss: 1.7372 - val_acc: 0.3797
Epoch 47/100
 - 42s - loss: 1.7242 - acc: 0.3470 - val_loss: 1.6234 - val_acc: 0.3872
Epoch 48/100
 - 41s - loss: 1.7237 - acc: 0.3474 - val_loss: 1.5752 - val_acc: 0.4095
Epoch 49/100
 - 42s - loss: 1.7143 - acc: 0.3527 - val_loss: 1.5845 - val_acc: 0.4057
Epoch 50/100
 - 41s - loss: 1.7174 - acc: 0.3509 - val_loss: 1.5196 - val_acc: 0.4335
Epoch 51/100
 - 42s - loss: 1.6883 - acc: 0.3631 - val

Epoch 74/100
 - 42s - loss: 1.6191 - acc: 0.3845 - val_loss: 1.5056 - val_acc: 0.4339
Epoch 75/100
 - 42s - loss: 1.6232 - acc: 0.3848 - val_loss: 1.4858 - val_acc: 0.4428
Epoch 76/100
 - 42s - loss: 1.6193 - acc: 0.3866 - val_loss: 1.5833 - val_acc: 0.4132
Epoch 77/100
 - 42s - loss: 1.6173 - acc: 0.3849 - val_loss: 1.4626 - val_acc: 0.4508
Epoch 78/100
 - 42s - loss: 1.6205 - acc: 0.3845 - val_loss: 1.5055 - val_acc: 0.4354
Epoch 79/100
 - 41s - loss: 1.6166 - acc: 0.3860 - val_loss: 1.5769 - val_acc: 0.4077
Epoch 80/100
 - 42s - loss: 1.6198 - acc: 0.3846 - val_loss: 1.5262 - val_acc: 0.4345
Epoch 81/100
 - 41s - loss: 1.6127 - acc: 0.3884 - val_loss: 1.5168 - val_acc: 0.4290
Epoch 82/100
 - 42s - loss: 1.6114 - acc: 0.3871 - val_loss: 1.5034 - val_acc: 0.4303
Epoch 83/100
 - 42s - loss: 1.6099 - acc: 0.3882 - val_loss: 1.5233 - val_acc: 0.4321
Epoch 84/100
 - 41s - loss: 1.6114 - acc: 0.3880 - val_loss: 1.4716 - val_acc: 0.4449
Epoch 85/100
 - 42s - loss: 1.6045 - acc: 0.3901 - val

IOError: ignored

In [4]:
import os
import cPickle, keras
os.environ["KERAS_BACKEND"] = "tensorflow"
import sys
import numpy as np
import matplotlib.pyplot as plt
from keras.layers.core import Reshape, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, ZeroPadding2D
from keras.regularizers import *
from keras.models import Model
from keras import layers
from keras.layers import Input, Dense, BatchNormalization
from sklearn.metrics import confusion_matrix
from sklearn.metrics import cohen_kappa_score, accuracy_score

name = 'ResNet'

# the following function is learning an approximation of the identity function from the data
# By trying to figure out that the output will be equal to input similar to Autoencoders
def identity_block(input_tensor, kernel_size, filters, stage, block):
    filters1, filters2, filters3 = filters
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = ZeroPadding2D((0, 2), data_format="channels_first")(input_tensor)
    x = Conv2D(filters1, (1, 5), name=conv_name_base + '2a',data_format="channels_first", init='glorot_uniform')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)
    x = Dropout(dr)(x)

    x = ZeroPadding2D((0, 2), data_format="channels_first")(x)
    x = Conv2D(filters2, kernel_size,
               padding='valid', name=conv_name_base + '2b',data_format="channels_first", init='glorot_uniform')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)
    x = Dropout(dr)(x)

    x = ZeroPadding2D((0, 2), data_format="channels_first")(x)
    x = Conv2D(filters3, (1, 3), name=conv_name_base + '2c',data_format="channels_first", init='glorot_uniform')(x)  # (1, 1)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)
    x = Dropout(dr)(x)

    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    return x


def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    """conv_block is the block that has a convolutionnal layer at shortcut"""
    filters1, filters2, filters3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'

    input_tensor_padding = ZeroPadding2D((0, 2), data_format="channels_first")(input_tensor)
    x = Conv2D(filters1, (1, 1),  # (1, 1)
               name=conv_name_base + '2a',data_format="channels_first",padding='valid', init='glorot_uniform')(input_tensor_padding)
    x = Activation('relu')(x)
    x = Dropout(dr)(x)

    x = ZeroPadding2D((0, 2), data_format="channels_first")(x)
    x = Conv2D(filters2, kernel_size, padding='valid',
               name=conv_name_base + '2b',data_format="channels_first", init='glorot_uniform')(x)
    x = Activation('relu')(x)
    x = Dropout(dr)(x)

    x = ZeroPadding2D((0, 2), data_format="channels_first")(x)
    x = Conv2D(filters3, (1, 3), name=conv_name_base + '2c',data_format="channels_first", padding='valid', init='glorot_uniform')(x)  # (1, 1)
    x = Dropout(dr)(x)

    shortcut = Conv2D(filters3, (1, 1),   # (1, 1)
                      name=conv_name_base + '1',data_format="channels_first", init='glorot_uniform')(input_tensor_padding)
    shortcut = Dropout(dr)(shortcut)

    x = layers.add([x, shortcut])
    x = Activation('relu')(x)
    return x

Xd = cPickle.load(open("RML2016.10a_dict (4).dat", 'rb'))
snrs, mods = map(lambda j: sorted(list(set(map(lambda x: x[j], Xd.keys())))), [1, 0])
X = []
lbl = []
for mod in mods:
    for snr in snrs:
        X.append(Xd[(mod, snr)])
        for i in range(Xd[(mod, snr)].shape[0]):  lbl.append((mod, snr))
X = np.vstack(X)
np.random.seed(2016)
n_examples = X.shape[0]
n_train = n_examples * 0.5
train_idx = np.random.choice(range(0, n_examples), size=int(n_train), replace=False)
test_idx = list(set(range(0, n_examples)) - set(train_idx))  # label
X_train = X[train_idx]
X_test = X[test_idx]

#here we will code the vectors into one hot vectors
def to_onehot(vec):
    vec_hot = np.zeros([len(vec), max(vec) + 1])
    vec_hot[np.arange(len(vec)), vec] = 1
    return vec_hot


trainy = list(map(lambda x: mods.index(lbl[x][0]), train_idx))
Y_train = to_onehot(trainy)
Y_test = to_onehot(list(map(lambda x: mods.index(lbl[x][0]), test_idx)))
in_shp = list(X_train.shape[1:])
print (X_train.shape, in_shp)
classes = mods
dr = 0.5
# Reshape [N,2,128] to [N,1,2,128] on input
input_x = Input(shape=(1,2,128))

x = conv_block(input_x, (1, 7), [50, 50, 50], stage=1, block='a')
x = identity_block(x, (1, 7), [50, 50, 50], stage=1, block='b')

layer_Flatten = Flatten()(x)
layer_dense1 = Dense(256, activation='relu', init='he_normal', name="dense1")(layer_Flatten)
layer_dropout = Dropout(dr)(layer_dense1)
layer_dense2 = Dense(len(classes), init='he_normal', name="dense2")(layer_dropout)
layer_softmax = Activation('softmax')(layer_dense2)
output = Reshape([len(classes)])(layer_softmax)

model = Model(inputs=input_x, outputs=output)
myadam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='categorical_crossentropy', optimizer=myadam, metrics=['accuracy'])
model.summary()

# Set up  params
epochs = 100  # number of epochs to train on
batch_size = 1024  # training batch size default 1024
filepath = "convmodrecnets_%s_0.5.wts.h5" % (name)

X_train = np.reshape(X_train, (-1,1,2,128))
X_test = np.reshape(X_test, (-1,1,2,128))

history = model.fit(X_train,
                    Y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=2,
                    validation_data=(X_test, Y_test))

plt.figure()
plt.title('Training performance')
plt.plot(history.epoch, history.history['loss'], label='train loss+error')
plt.plot(history.epoch, history.history['val_loss'], label='val_error')
plt.legend()
plt.savefig('%s Training performance' %(name))
plt.show()

model.load_weights(filepath)
score = model.evaluate(X_test, Y_test, verbose=0, batch_size=batch_size)
print('evaluate_score:', score)